In [ ]:
%matplotlib inline
import sys, os
import numpy as np
import inspect
from openxrd import find_peaks_2d
from openxrd import find_peaks_1d
from openxrd import find_saddle_1d
from openxrd import fit_single
from openxrd import get_fit_all_1d
from openxrd import get_fit_all_2d
from openxrd import fits_to_csv_multitype
from openxrd import fit_data_to_csv
from openxrd import BrukerData
from openxrd import fit_multipeak
from openxrd import fits_to_csv
from openxrd import csv_append_col

from datasetmetta import get_name_data

import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.patheffects as path_effects
from mpl_toolkits.axes_grid1 import make_axes_locatable

import csv
from lmfit import models
from lmfit import lineshapes
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from scipy import signal
import scipy

from scipy.signal import argrelextrema
#PLOT Setup

axlabels = {'family':               'sans',
            'color':                'black',
            'weight':               'normal',
            'size':                 20,
            }
titles = {'family':                 'sans',
          'color':                  'black',
          'weight':                 'normal',
          'size':                   24,
          }
labels = {'family':                 'sans',
          'fontname':               'DejaVu Sans',
          'color':                  '#000000', #'#66ff33',
          'weight':                 'normal',
          'size':                   14,
          'verticalalignment':      'center',
          'horizontalalignment':    'right'
          }
ledg =   {'size':                    14,
          'family':                  'sans'
         }
def butter_lp_filter(y, cutoff=None, order=2):
    """
    Performs a low pass butterworth filter on the data with cutoff
    frequency that defaults to 2/len(y)
    Arguments
    ---------
    cutoff (float) [default: 2/len(y)]
        cutoff frequency at which to filter the data
    order (int) [default: 2]
        filter order
    Returns
    -------
    (array)
        low pass filtered array same size as data
        
    """
    
    if cutoff is None:
        cutoff = 2 / len(y)
    B, A = signal.butter(order, cutoff, output='ba')
    return signal.filtfilt(B, A, y)

def butter_hp_filter(y, cutoff=None, order=2):
    
    if cutoff is None:
        cutoff = 2 / len(y)
    B, A = signal.butter(order, cutoff, btype='high')
    return signal.filtfilt(B, A, y)

def find_peaks(y, y_r, y_sd, data, sigma=3, neigh_multi=0.01, neigh_add=5):
    neighbors = int(y.size * neigh_multi) + neigh_add
    (y_mx,) = argrelextrema(y, np.greater_equal, order=neighbors, mode='clip')
    #print(y_mx)
    (peaks,_) = data.get_real_xy(y_mx.tolist())
    #print('peaks', peaks)
    #for i, p in enumerate(y_mx):
    #    if y[p] > y_r[p] +sigma*y_sd[p]:
    #        print(peaks[i], y_r[p], y_sd[p], y[p], y_r[p] + sigma* y_sd[p])
            
    y_mx = [p for p in y_mx if y[p] > y_r[p] + sigma* y_sd[p]]
    (peaks,_) = data.get_real_xy(y_mx)
    #print(peaks)
    return peaks
    #(peaks,_) = data.get_real_xy(peaks)
    #peaks = [round(r) for r in peaks]
    
    
        #def find_peaks(ax, x, y, data=None, neigh=None, threshold=1000, peak_widths=(10, 25)):
        #print(len(y), max(x))
        #peak_widths = (len(y)/max(x)/2, len(y)/max(x)*10)
        #print(peak_widths)
        #get peaks in data
        #peaks = find_peaks_cwt(y, neigh)
        #if neigh:
        #    data_max = np.asarray(find_peaks_1d(y, neigh[0], neigh[1]))
        #    data_min = np.asarray(find_saddle_1d(y, neigh[0], neigh[1]))
        #else:
        #    data_max = np.asarray(find_peaks_1d(y))
        #    data_min = np.asarray(find_saddle_1d(y))
        #maxima = (y == data_max)
        #diff = ((data_max - data_min) > threshold)
        #maxima[diff == 0] = 0
        #peaks = [m for m in x if maxima[m]]
        #peaks = (signal.find_peaks_cwt(y, peak_widths)).tolist()
        #print(peaks)
        #(peaks,_) = data.get_real_xy(peaks)
        #peaks = [round(r) for r in peaks]
        #print(peaks)

def get_pdf(pdffile, peaks, ax, x, y):
        # fastest?
        # https://softwarerecs.stackexchange.com/questions/7463/fastest-python-library-to-read-a-csv-file
        with open(pdffile, 'r') as f:
            pdfs = csv.reader(row for row in f if not
                               row.startswith('#'))
            peaks = [round(p, 1) for p in peaks]
            print(peaks)
            ##change to find closest loop through peaks
            for pd in pdfs: # run in order to only select best #sorted(pdfs,key=lambda x: x[0]):
                x_p = round(float(pd[0]),1)
                if (x.min() < x_p < x.max()):
                    #print(x_p)
                    rn = np.linspace(-0.5, 0.5, 11)
                    #print(rn+x_p)
                    if any(round(x, 1) in peaks for x in rn+x_p):
                        del peaks[np.abs(np.asarray(peaks)-x_p).argmin()]
                        amp = y.max() * float(pd[1]) / 100.0
                    
                        txt = ax.text(x_p, 0.1, pd[2],
                                      fontdict=labels, rotation=90)
                        # Make the text fancy
                        #txt.set_path_effects([path_effects.Stroke(linewidth=1,
                        #                     foreground='black'),
                        #                     path_effects.Normal()])
                        # Add a line
                        line = ax.plot([x_p, x_p],[0.01, amp], 
                                       linestyle=':', color='black')
        print(peaks)

def smooth(y, box_pts, poly=None):
    if poly:
        y_smooth = savgol_filter(y, box_pts, poly)
    else:
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


def get_background(x, y, neigh_multi=0.01, neigh_add=0, cutper = 2):
    neighbors = int(y.size * neigh_multi) + neigh_add
    y_r = np.copy(y)
    y_sd = np.zeros(len(y))
    i = 0
    while i+2*neighbors < len(y):
        mean1 = np.mean(y_r[i:i+neighbors])
        sd1 = y_sd[i]
        #print(sd1)
        if sd1 == 0:
            sd1 = np.std(y[i:i+neighbors])
            y_sd[i:i+neighbors] = sd1
        # removing outlires > 2std     
        y_r[i:i+neighbors] = [c if c < mean1+2*sd1 else mean1 for c in y_r[i:i+neighbors]]
        
        mean2 = np.mean(y_r[i+neighbors:i+2*neighbors])
        sd2 = np.std(y_r[i+neighbors:i+2*neighbors])
        #print(sd1, sd2)
        if mean2 > cutper*mean1 or sd2 > cutper*sd1:
            y_r[i+neighbors:i+2*neighbors] = mean1
            y_sd[i+neighbors:i+2*neighbors] = sd1
        else:
            y_sd[i+neighbors:i+2*neighbors] = sd2
        i += neighbors

    return y_r, y_sd

def main(files, pdffile, name=None, peak_add=None, peak_sub=None):
    fig = plt.figure(figsize=(10, 6))#, dpi=100)
    ax = fig.add_subplot(111)
    ax.set_title(name, fontdict=titles, y=1)
    #axis Lables
    ax.set_xlabel('2\u03b8[\u00b0]', fontdict=titles)
    # ax.set_xlabel('\u03c9[\u00b0]', fontdict=titles) #RC
    ax.set_ylabel(u'Intensity [arbitrary units]', fontdict=titles)
    # Set axis tick labels font
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        for prop in axlabels:
            getattr(label, 'set_' + prop)(axlabels[prop])
    for file in files:
        data = BrukerData(file)
        #print(len(data.rngs))
        for j, rng in enumerate(data.rngs):
            #print (j)
            twoth_0 = rng.metta['start_2th']
            twoth_s = rng.metta['step_size']
            twoth_l = rng.metta['steps']
            twoth_e = twoth_0 + twoth_s*twoth_l
            x = np.linspace(twoth_0, twoth_e, twoth_l) 
            if j == 0:
                ax.set_xlim([min(x), max(x)])
            y = np.asarray(rng.counts_data)
            #print(len(y))
            y[y< 1] = 1
            d_x = rng.metta['drive_x']
            d_y = rng.metta['drive_y']
            d_r = rng.metta['drive_phi']
            ax.semilogy(x, y, label='({0:.0f},{1:.0f}) {2:.0f}\u00b0'.format(d_x, d_y, d_r)) #plot
            ax.set_ylim(bottom=0.01, top=max(y)*10)
            ax.set_yticklabels([])
            # plot smoothed if using
            #if smo:
            #    y = smooth(y, smo[0], smo[1])
            #    ax.plot(x, y) # window size 51, polynomial order 3
            if j == 0:    
                # Fit background with lmfit and compair to background to determine if true peak    
                # first remove all points above the average
                y_r, y_sd = get_background(x, y)
                #ax.plot(x, y_r, 'r-')
            
                # Plot StD
                #ax2 = ax.twinx()
                #ax2. plot(x, y_sd, 'g-')
                # smooth (fit) StD
                mod = models.PolynomialModel(3)
                pars = mod.guess(y_sd, x=x)
                out = mod.fit(y_sd, pars, x=x)
                y_sd = out.best_fit
                # Plot SStd
                #ax2. plot(x, y_sd, 'g:')
            
                # find background function
                mod = models.PolynomialModel(3)
                pars = mod.guess(y_r, x=x)
                out = mod.fit(y_r, pars, x=x)
                y_r = out.best_fit
                # Plot Background function
                #ax.plot(x, y_r, 'r-')
            
            
                peaks = find_peaks(y, y_r, y_sd, data, sigma=4)
                if peak_add is not None:
                    peaks = np.concatenate(peaks, peak_add)
                if peak_sub is not None:
                    for pk in peak_sub:
                         del peaks[np.abs(np.asarray(peaks)-pk).argmin()]

                #ax.plot(x, y_r, 'b:')
                get_pdf(pdffile, peaks, ax, x, y)

            # Maybe look more into this in the future...
            # ax.plot(x, scipy.fftpack.fft(y), 'g-')

            #ax.plot(x, butter_lp_filter(y,0.1,3))
            #ax.plot(x, butter_hp_filter(butter_lp_filter(y,0.01,3),.0000019,3))
            
            #get_pdf(pdffile, ax, x, y, data, neigh)
            
    plt.legend(#bbox_to_anchor=(1., 1.02, 1., .102),
           loc= 9, ncol=5, mode="expand", borderaxespad=0., prop=ledg)        
    plt.tight_layout()
    plt.show()


# Get data
root = '/mnt/W/Austin_Fox/'
rootf = root + 'XRD/'

In [ ]:
name = '5582'
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
y =main(files, pdffile, name)

In [ ]:
name = '6405'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '7050'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '6439'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '7526'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '8718_full'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '8720_fastmap'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '8723_full_map'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = 'S2_P1_Al'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[40.1])

In [ ]:
name = 'S2_P2_Si'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name, peak_sub=[74.0])

In [ ]:
name = 'S3_P3_Al'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = 'S3_P4_TOx'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
name = '8720_fastmap'
print(name)
files = [rootf + name +'.raw']
pdffile = root + 'devel/BAW_peaks.csv'
main(files, pdffile, name)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.2
yhat = savgol_filter(y, 51, 3) # window size 51, polynomial order 3

plt.plot(x,y)
plt.plot(x,yhat, color='red')
plt.show()

In [ ]:
import numpy as np
import scipy.fftpack

N = 100
x = np.linspace(0,2*np.pi,N)
y = np.sin(x) + np.random.random(N) * 0.2

w = scipy.fftpack.rfft(y)
f = scipy.fftpack.rfftfreq(N, x[1]-x[0])
spectrum = w**2

cutoff_idx = spectrum < (spectrum.max()/5)
w2 = w.copy()
w2[cutoff_idx] = 0

y2 = scipy.fftpack.irfft(w2)

plt.plot(x,y)
plt.plot(x,y2, color='red')
plt.show()

In [ ]:
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array

def peakdet(v, delta, x = None):
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html
    
    Returns two arrays
    
    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.
    
    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.
    
    """
    maxtab = []
    mintab = []
       
    if x is None:
        x = arange(len(v))
    
    v = asarray(v)
    
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    
    lookformax = True
    
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True

    return array(maxtab), array(mintab)

if __name__=="__main__":
    import numpy as np
    import matplotlib.pyplot as plt
    series = [0,0,0,2,0,0,0,-2,0,0,0,2,0,0,0,-2,0]
    series = y
    maxtab, mintab = peakdet(series,50)
    plt.semilogy(series)
    plt.scatter(array(maxtab)[:,0], array(maxtab)[:,1], color='blue')
    plt.scatter(array(mintab)[:,0], array(mintab)[:,1], color='red')
    plt.show()